In [1]:
import time
import re
import numpy as np
import pandas as pd
# from src.lib.utils import open_html, get_driver, popup_scroll_down, scroll_down, insert_point
# from src.lib.job_plannet_utils import close_popup

from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from user_agent import generate_user_agent

def get_driver(url, driver):
    while(1):
        try:    
            driver.get(url)
            break
        except:
            continue    

def insert_point(src):
    return src.replace(' ','.')

def close_popup(driver):
    # 팝업창으로 드라이브 스위칭
    path = 'ab-iam-root v3 ab-animate-in ab-animate-out ab-effect-html ab-show'
    popup = driver.find_element(By.CLASS_NAME, path.replace(' ','.'))
    pop_driver = popup.find_element(By.TAG_NAME, 'iframe')
    driver.switch_to.frame(pop_driver)

    # 팝업창 닫기
    src = insert_point('bz-pin-row-section bz-pin-row-bottom-section')
    button_box = driver.find_element(By.CLASS_NAME, src)
    button_box.find_element(By.CLASS_NAME, 'bz-column')

    close_button = button_box.find_element(By.CLASS_NAME, 'bz-column')
    close_button.click()

    # 드라이브 원위치
    driver.switch_to.default_content()


def input_text_to_search_box(search_bar, text, by, src):
    search_bar.click()
    search_box = search_bar.find_element(by, src)
    search_box.send_keys(text)
    search_box.send_keys(Keys.ENTER)

def find_job_list(driver):
    job_list = driver.find_element(By.CLASS_NAME, 'job_search_list')
    job_list = job_list.find_element(By.CLASS_NAME, 'list')
    job_list = job_list.find_elements(By.TAG_NAME, 'li')
    return job_list

def move_points(driver, src, x, y):
    left_point, right_point = src.find_elements(By.CLASS_NAME, insert_point('ui-slider-handle ui-state-default ui-corner-all'))
    move_offset = x*30
    actions = ActionChains(driver)
    actions.click_and_hold(left_point).move_by_offset(move_offset, 0)
    actions.release()
    actions.perform()
    move_offset = y*30
    actions = ActionChains(driver)
    actions.click_and_hold(right_point).move_by_offset(move_offset, 0)
    actions.release()
    actions.perform()

driver = webdriver.Chrome()
job_korea_url = 'https://www.jobplanet.co.kr/welcome/index'
get_driver(job_korea_url, driver)

try:
    close_popup(driver)
except:
    print('There is no popup window.')

search_bar = driver.find_element(By.CLASS_NAME, insert_point('schbar_box'))
input_text_to_search_box(search_bar, 'AI', By.TAG_NAME, 'input')

src = 'group desktop'
gdesk = driver.find_element(By.CLASS_NAME, insert_point(src))
src = 'inline-flex items-center'

# 0 : 관련 기업 전체, 1 : 채용 중인 기업
more_view = gdesk.find_elements(By.CLASS_NAME, insert_point(src))[1]
sub_url = more_view.get_attribute('href')

# driver switching
main_window = driver.current_window_handle
driver.execute_script("window.open('');")
all_windows = driver.window_handles
driver.switch_to.window(all_windows[-1])
driver.get(sub_url)

search_box = driver.find_element(By.CLASS_NAME, insert_point('jply_ip_item_box'))
input_text_to_search_box(search_box,'AI', By.ID, 'jp_job_title')

data = {
    '기업명' : [],
    '요약' : [],
    '기업 소개' : [],
    '주요 업무' : [],
    '자격 요건' : [],
    '우대사항' : [],
    '채용 절차' : [],
    '복지 및 혜택' : [],
    '회사위치' : [],
}

key_list = list(data.keys())

pag_bar = driver.find_element(By.CLASS_NAME, 'jply_pagination_ty1')
pag_bar = pag_bar.find_elements(By.TAG_NAME, 'button')
max_page = int(pag_bar[-2].text)

for i in tqdm(range(max_page-1)):
    # 페이지 별 회사 리스트 가져오기
    job_list = find_job_list(driver)
    
    time.sleep(1)
    for j in range(len(job_list)):
        time.sleep(1)        
        job_list[j].find_element(By.TAG_NAME, 'a').click()

       # 회사명
        check_data = {key:'-' for key in data.keys()}

        name = driver.find_element(By.CLASS_NAME, 'jobs_company').text
        check_data['기업명'] = name
        boxes = driver.find_element(By.CLASS_NAME, 'recruitment-detail')
        boxes = boxes.find_elements(By.CLASS_NAME, 'recruitment-detail__box')

        for box in boxes[:-1]:
            key = box.text.split('\n')[0]
            if key in key_list:
                check_data[key] = box.text

        for key, value in check_data.items():
            data[key].append(value)

        time.sleep(3)
        
    # 페이지 넘기기
    pag_bar = driver.find_element(By.CLASS_NAME, 'jply_pagination_ty1')
    pag_bar = pag_bar.find_elements(By.TAG_NAME, 'button')
    if i<=2:
        pag_bar[i+1].click()
    elif i>=max_page-1:
        pag_bar[-(18-i)-1].click()
    else:
        pag_bar[4].click()

    time.sleep(2)


There is no popup window.


100%|██████████| 16/16 [13:09<00:00, 49.33s/it]


In [2]:
data = pd.DataFrame(data)
data

,기업명,요약,기업 소개,주요 업무,자격 요건,우대사항,채용 절차,복지 및 혜택,회사위치
0,씨제이올리브네트웍스(주),"요약\n마감일\n2024.10.27 D-28\n직무\n사업 기획, 전략 기획, 서비...","기업 소개\n고객과 사회의 행복을 선도하는 Top Tier 디지털 서비스 기업, C...",주요 업무\nCJ그룹 AI실은 CJ 주식회사 산하에서 AI기술을 비즈니스에 융합하여...,자격 요건\n- 학사 이상\n- 관련 경력 7년 이상\n└ 전략기획 및 컨설팅 직무...,"우대사항\n- AI 및 컴퓨터공학, 소프트웨어공학 등 관련 분야 전공 또는 경력\n...",채용 절차\n[전형절차] 서류전형 > 1차(PT)면접 > 2차(PT)면접 > 3차면...,-,"회사위치\n서울 용산구 한강대로 366, 10층"
1,씨제이올리브네트웍스(주),"요약\n마감일\n2024.10.27 D-28\n직무\n사업 기획, 전략 기획, 서비...","기업 소개\n고객과 사회의 행복을 선도하는 Top Tier 디지털 서비스 기업, C...",주요 업무\nCJ그룹 AI실은 CJ 주식회사 산하에서 AI기술을 비즈니스에 융합하여...,자격 요건\n- 학사 이상\n- 관련 경력 7년 이상\n└ 전략기획 및 컨설팅 직무...,"우대사항\n- AI 및 컴퓨터공학, 소프트웨어공학 등 관련 분야 전공 또는 경력\n...",채용 절차\n[전형절차] 서류전형 > 1차(PT)면접 > 2차(PT)면접 > 3차면...,-,"회사위치\n서울 용산구 한강대로 366, 10층"
2,씨제이올리브네트웍스(주),"요약\n마감일\n2024.10.31 D-32\n직무\n소프트웨어 개발, 네트워크/보...",기업 소개\n[회사소개]\n아하랩스는 'AI 기술로 모두의 일상을 이상적인 미래로 ...,주요 업무\n자사에서 개발한 Smart Factory Solution의 AI 모델을...,"자격 요건\n• 컴퓨터 공학, 통계학 등 머신 러닝 관련 분야를 전공하신 분\n• ...",우대사항\n• ML 분야 박사 이상 또는 이에 준하는 경력을 보유하신 분\n• 모델...,채용 절차\n• 서류 전형 – 코딩테스트(온라인) – 실무면접 – 임원면접 – 채용...,복지 및 혜택\n□ 시차출퇴근제도\n- 오전 8시~ 11시 사이 자유롭게 출근하세요...,"회사위치\n경기도 성남시 분당구 성남대로 43번길 10, 7층 712호(구미동, 하..."
3,씨제이올리브네트웍스(주),"요약\n마감일\n2024.10.31 D-32\n직무\n서비스 기획, 사업 기획, 전...","기업 소개\nAI OCR, 3D 스타트업\n\n[우리의 스토리]\n\n시작\nKDL...","주요 업무\n*콘텐츠 마케팅, 카피라이터, PR\n\n- 기업 고객 타겟 콘텐츠 기...",자격 요건\n- 콘텐츠 마케팅 글쓰기 경력 3년 이상\n- 기술 이해도: IT 기술...,우대사항\n- B2B 콘텐츠 마케팅 경험: B2B 기업 대상 콘텐츠 마케팅 경험이 ...,-,-,회사위치\n서울 서초구 강남대로 89길 30 르헤브빌딩
4,씨제이올리브네트웍스(주),요약\n마감일\n2024.12.05 D-67\n직무\nPM/PO\n경력\n3 ~ 1...,기업 소개\n큰 잠재력을 가진 글로벌 디자인 산업에서 '디자인-인쇄-수익화' 가 결...,"주요 업무\n| 주요 프로덕트\n""미리캔버스는 이런 프로덕트에 집중하고 있어요""\n...","자격 요건\n""이런 분이라면 목표 달성에 확신을 얻을 것 같아요""\n\n• AI와 ...","우대사항\n""이런 분이라면 장기적으로 서로에게 더 긍정적일 것 같아요!""\n\n• ...",채용 절차\n• [서류 전형] - [1차 인터뷰] - [2차 인터뷰] - [처우 협...,"복지 및 혜택\n""미리디는 복지를 통해 나눔의 경영을 실현하고 있습니다.""\n\n[...","회사위치\n서울시 구로구 디지털로31길 12, 8층 (구로동, 태평양물산)"
...,...,...,...,...,...,...,...,...,...
155,(주)스텝페이,요약\n마감일\n2024.11.28 상시 채용\n직무\nPM/PO\n경력\n경력무관...,기업 소개\n* 메이아이는 이런 기업입니다 *\n\nmake offline smar...,주요 업무\n영상 처리 인공지능 스타트업 메이아이(mAy-I)에서 Project M...,자격 요건\n메이아이의 Project Manager는 고객사 및 다양한 직군과의 소...,"우대사항\n지원을 위한 필수 조건은 아니나, 보유하고 계시면 업무를 수행하실 때 큰...",채용 절차\n■ 직전 연봉 대비 협상이 가능합니다.\n\n\n\n* 이런 만남이 준...,복지 및 혜택\n□ 최대의 자유 속에서 최고의 효율을 냅니다\n• 핫타임 제도 : ...,"회사위치\n서울 강남구 삼성로 85길 26, V&S빌딩 5층"
156,(주)스텝페이,요약\n마감일\n2024.10.26 상시 채용\n직무\n빅데이터 엔지니어\n경력\n...,기업 소개\n재능은 누구나 가지고 있지만 재능을 실현하기 위해 걸어야 할 어려운 과...,주요 업무\n[IP 빅데이터 분석팀은 어떤 팀인가요?]\n\n- IP 빅데이터 분석...,자격 요건\n• 특허조사분석 또는 빅데이터분석 경력 3년 이상,우대사항\n• 새로운 IP빅데이터분석에 대한 관심과 열정으로 함께 성장시킬 의지가 ...,-,복지 및 혜택\n※ 워트 멤버들의 성장을 위한 지원을 계속해서 확대해나가고 있는 중...,"회사위치\n서울 서초구 강남대로 507, 6층"
157,(주)스텝페이,"요약\n마감일\n2024.11.18 D-50\n직무\n국내영업, 영업기획/관리/지원...","기업 소개\n""리워드 기반의 애드테크 No.1 기업, 버즈빌""\n\n마케팅부터 수익...",주요 업무\n[Who we are]\nAI 기반의 리워드 광고 플랫폼 기업 버즈빌은...,자격 요건\n• 학사학위 이상의 학위를 소지하신 분\n• 3년 이상 모바일 광고 세...,우대사항\n• 미디어렙 세일즈 관련 업무 경험이 있으신 분,채용 절차\n• 서류전형: 경력기술 중심의 이력서\n• 1차 인터뷰(Job-fit)...,복지 및 혜택\n🧐 몰입을 위한 최적의 환경을 제공합니다.\n• 주 1회 전사 재택...,"회사위치\n서울특별시 송파구 석촌호수로 272 MY빌딩 2, 3, 4, 5층"
158,(주)스텝페이,"요약\n마감일\n2024.11.27 상시 채용\n직무\n그래픽디자인, 애니메이션디자...","기업 소개\n""누구나 큰일 낼 수 있어""\n팀스파르타는 모두가 저마다 가슴 뛰는 자...","주요 업무\n🚀 로켓단게임즈를 소개해요\n• 로켓단게임즈는 '주술사 키우기', '타...",자격 요건\n🔎 이런 분을 찾고 있어요\n• Unity 엔진을 활용하여 게임을 제작...,"우대사항\n👍 이런 분이라면 더 좋아요\n• 유니티 에셋을 활용하여, 베리에이션 및...",채용 절차\n🚢 합류 여정을 안내드려요\n• 서류 전형 > 실무 인터뷰 > 컬쳐핏 ...,복지 및 혜택\n✔️ 몰입을 위해 지원합니다\n서로에 대한 신뢰를 바탕으로 설계된 ...,"회사위치\n서울시 강남구 테헤란로 44길 8, 12층 & 13층 (아이콘역삼빌딩)"


In [3]:
import mysql.connector as mc

remote = mc.connect(
    host="roseda.cj6s66ayq6e8.ap-northeast-2.rds.amazonaws.com",
    user="fo",
    password="1234",
    database='folium')

cur = remote.cursor(buffered=True)

input_data = ', '.join([f'{name.replace(' ','_')} VARCHAR(255)' for name in data.columns])
create_table = f'CREATE TABLE recruit_info2 ({input_data})'
Insert_info = 'INSERT INTO recruit_info2 VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)'
cur.execute(create_table)

for i, row in data.iterrows():
    cur.execute(Insert_info, tuple(row))
    remote.commit()

cur.execute('SELECT * FROM recruit_info2')
result = cur.fetchall()
for i, row in enumerate(result):
    if i == 10:
        break
    print(row)

remote.close()

('씨제이올리브네트웍스(주)', '요약\n마감일\n2024.10.27 D-28\n직무\n사업 기획, 전략 기획, 서비스 기획\n경력\n7 ~ 12년\n고용형태\n정규직\n근무지역\n서울\n스킬\n기획, 서비스기획, 사업관리, 전략기획, 사업기획, 사업개발', '기업 소개\n고객과 사회의 행복을 선도하는 Top Tier 디지털 서비스 기업, CJ올리브네트웍스!\n\nCJ올리브네트웍스는 1995년 창립이래 제조, 유통, 물류, 미디어 등 생활∙문화 기반의 IT서비스를 성공적으로 제공함으로써 고객 만족과 가치를 실현하고 있으며, 고객의 신뢰와 지지를 바탕으로 견실하고 안정적인 성장을 지속해오고 있습니다.\n\nIT시스템 구축 및 서비스 제공을 통해 축적된 경험과 역량을 바탕으로 오픈소스(Open Source), 빅데이터(B', '주요 업무\nCJ그룹 AI실은 CJ 주식회사 산하에서 AI기술을 비즈니스에 융합하여 새로운 가치를 만들어내고 혁신적인 성과 창출을 위해 AI핵심 기술을 기반으로 CJ 계열사의 사업 성장에 기여하는 CJ 그룹 조직입니다. 기술이 생활 문화 발전과 창조에 기여하는 시대에, 데이터 분석 및 인공지능 기술을 기반으로 생활과 문화 발전/개선을 위한 AI연구와 미래 산업에 필요한 AI Core Technology 확보를 수행합니다.\n\nAI Tech Strategy&', '자격 요건\n- 학사 이상\n- 관련 경력 7년 이상\n└ 전략기획 및 컨설팅 직무 경험\n- 원활한 커뮤니케이션 능력 및 협업 능력\n- 새로운 기술, 플랫폼에 대한 관심과 열정', '우대사항\n- AI 및 컴퓨터공학, 소프트웨어공학 등 관련 분야 전공 또는 경력\n- 대규모 AI 프로젝트 개발 또는 관리 경험\n- 기술 로드맵 개발 및 실행 경험\n- 기술 컨설팅 또는 스타트업, 지주사 근무 경험\n- 영어 커뮤니케이션 능통자 또는 글로벌 프로젝트 경험', '채용 절차\n[전형절차] 서류전형 > 1차(PT)면접 > 2차(PT)면접 > 3차면접 or 티미팅 > 인성검사/평판조회 

In [4]:
import mysql.connector as mc

data = {
    '기업명' : [],
    '요약' : [],
    '기업 소개' : [],
    '주요 업무' : [],
    '자격 요건' : [],
    '우대사항' : [],
    '채용 절차' : [],
    '복지 및 혜택' : [],
    '회사위치' : [],
}

col = list(data.keys())
remote = mc.connect(
    host="roseda.cj6s66ayq6e8.ap-northeast-2.rds.amazonaws.com",
    user="fo",
    password="1234",
    database='folium')

cur = remote.cursor(buffered=True)

cur.execute('SELECT * FROM recruit_info2')
result = cur.fetchall()
for row in result:
    for i, item in enumerate(row):
        data[col[i]].append(item)

remote.close()

In [6]:
data = pd.DataFrame(data)
data

,기업명,요약,기업 소개,주요 업무,자격 요건,우대사항,채용 절차,복지 및 혜택,회사위치
0,씨제이올리브네트웍스(주),"요약\n마감일\n2024.10.27 D-28\n직무\n사업 기획, 전략 기획, 서비...","기업 소개\n고객과 사회의 행복을 선도하는 Top Tier 디지털 서비스 기업, C...",주요 업무\nCJ그룹 AI실은 CJ 주식회사 산하에서 AI기술을 비즈니스에 융합하여...,자격 요건\n- 학사 이상\n- 관련 경력 7년 이상\n└ 전략기획 및 컨설팅 직무...,"우대사항\n- AI 및 컴퓨터공학, 소프트웨어공학 등 관련 분야 전공 또는 경력\n...",채용 절차\n[전형절차] 서류전형 > 1차(PT)면접 > 2차(PT)면접 > 3차면...,-,"회사위치\n서울 용산구 한강대로 366, 10층"
1,씨제이올리브네트웍스(주),"요약\n마감일\n2024.10.27 D-28\n직무\n사업 기획, 전략 기획, 서비...","기업 소개\n고객과 사회의 행복을 선도하는 Top Tier 디지털 서비스 기업, C...",주요 업무\nCJ그룹 AI실은 CJ 주식회사 산하에서 AI기술을 비즈니스에 융합하여...,자격 요건\n- 학사 이상\n- 관련 경력 7년 이상\n└ 전략기획 및 컨설팅 직무...,"우대사항\n- AI 및 컴퓨터공학, 소프트웨어공학 등 관련 분야 전공 또는 경력\n...",채용 절차\n[전형절차] 서류전형 > 1차(PT)면접 > 2차(PT)면접 > 3차면...,-,"회사위치\n서울 용산구 한강대로 366, 10층"
2,씨제이올리브네트웍스(주),"요약\n마감일\n2024.10.31 D-32\n직무\n소프트웨어 개발, 네트워크/보...",기업 소개\n[회사소개]\n아하랩스는 'AI 기술로 모두의 일상을 이상적인 미래로 ...,주요 업무\n자사에서 개발한 Smart Factory Solution의 AI 모델을...,"자격 요건\n• 컴퓨터 공학, 통계학 등 머신 러닝 관련 분야를 전공하신 분\n• ...",우대사항\n• ML 분야 박사 이상 또는 이에 준하는 경력을 보유하신 분\n• 모델...,채용 절차\n• 서류 전형 – 코딩테스트(온라인) – 실무면접 – 임원면접 – 채용...,복지 및 혜택\n□ 시차출퇴근제도\n- 오전 8시~ 11시 사이 자유롭게 출근하세요...,"회사위치\n경기도 성남시 분당구 성남대로 43번길 10, 7층 712호(구미동, 하..."
3,씨제이올리브네트웍스(주),"요약\n마감일\n2024.10.31 D-32\n직무\n서비스 기획, 사업 기획, 전...","기업 소개\nAI OCR, 3D 스타트업\n\n[우리의 스토리]\n\n시작\nKDL...","주요 업무\n*콘텐츠 마케팅, 카피라이터, PR\n\n- 기업 고객 타겟 콘텐츠 기...",자격 요건\n- 콘텐츠 마케팅 글쓰기 경력 3년 이상\n- 기술 이해도: IT 기술...,우대사항\n- B2B 콘텐츠 마케팅 경험: B2B 기업 대상 콘텐츠 마케팅 경험이 ...,-,-,회사위치\n서울 서초구 강남대로 89길 30 르헤브빌딩
4,씨제이올리브네트웍스(주),요약\n마감일\n2024.12.05 D-67\n직무\nPM/PO\n경력\n3 ~ 1...,기업 소개\n큰 잠재력을 가진 글로벌 디자인 산업에서 '디자인-인쇄-수익화' 가 결...,"주요 업무\n| 주요 프로덕트\n""미리캔버스는 이런 프로덕트에 집중하고 있어요""\n...","자격 요건\n""이런 분이라면 목표 달성에 확신을 얻을 것 같아요""\n\n• AI와 ...","우대사항\n""이런 분이라면 장기적으로 서로에게 더 긍정적일 것 같아요!""\n\n• ...",채용 절차\n• [서류 전형] - [1차 인터뷰] - [2차 인터뷰] - [처우 협...,"복지 및 혜택\n""미리디는 복지를 통해 나눔의 경영을 실현하고 있습니다.""\n\n[...","회사위치\n서울시 구로구 디지털로31길 12, 8층 (구로동, 태평양물산)"
...,...,...,...,...,...,...,...,...,...
155,(주)스텝페이,요약\n마감일\n2024.11.28 상시 채용\n직무\nPM/PO\n경력\n경력무관...,기업 소개\n* 메이아이는 이런 기업입니다 *\n\nmake offline smar...,주요 업무\n영상 처리 인공지능 스타트업 메이아이(mAy-I)에서 Project M...,자격 요건\n메이아이의 Project Manager는 고객사 및 다양한 직군과의 소...,"우대사항\n지원을 위한 필수 조건은 아니나, 보유하고 계시면 업무를 수행하실 때 큰...",채용 절차\n■ 직전 연봉 대비 협상이 가능합니다.\n\n\n\n* 이런 만남이 준...,복지 및 혜택\n□ 최대의 자유 속에서 최고의 효율을 냅니다\n• 핫타임 제도 : ...,"회사위치\n서울 강남구 삼성로 85길 26, V&S빌딩 5층"
156,(주)스텝페이,요약\n마감일\n2024.10.26 상시 채용\n직무\n빅데이터 엔지니어\n경력\n...,기업 소개\n재능은 누구나 가지고 있지만 재능을 실현하기 위해 걸어야 할 어려운 과...,주요 업무\n[IP 빅데이터 분석팀은 어떤 팀인가요?]\n\n- IP 빅데이터 분석...,자격 요건\n• 특허조사분석 또는 빅데이터분석 경력 3년 이상,우대사항\n• 새로운 IP빅데이터분석에 대한 관심과 열정으로 함께 성장시킬 의지가 ...,-,복지 및 혜택\n※ 워트 멤버들의 성장을 위한 지원을 계속해서 확대해나가고 있는 중...,"회사위치\n서울 서초구 강남대로 507, 6층"
157,(주)스텝페이,"요약\n마감일\n2024.11.18 D-50\n직무\n국내영업, 영업기획/관리/지원...","기업 소개\n""리워드 기반의 애드테크 No.1 기업, 버즈빌""\n\n마케팅부터 수익...",주요 업무\n[Who we are]\nAI 기반의 리워드 광고 플랫폼 기업 버즈빌은...,자격 요건\n• 학사학위 이상의 학위를 소지하신 분\n• 3년 이상 모바일 광고 세...,우대사항\n• 미디어렙 세일즈 관련 업무 경험이 있으신 분,채용 절차\n• 서류전형: 경력기술 중심의 이력서\n• 1차 인터뷰(Job-fit)...,복지 및 혜택\n🧐 몰입을 위한 최적의 환경을 제공합니다.\n• 주 1회 전사 재택...,"회사위치\n서울특별시 송파구 석촌호수로 272 MY빌딩 2, 3, 4, 5층"
158,(주)스텝페이,"요약\n마감일\n2024.11.27 상시 채용\n직무\n그래픽디자인, 애니메이션디자...","기업 소개\n""누구나 큰일 낼 수 있어""\n팀스파르타는 모두가 저마다 가슴 뛰는 자...","주요 업무\n🚀 로켓단게임즈를 소개해요\n• 로켓단게임즈는 '주술사 키우기', '타...",자격 요건\n🔎 이런 분을 찾고 있어요\n• Unity 엔진을 활용하여 게임을 제작...,"우대사항\n👍 이런 분이라면 더 좋아요\n• 유니티 에셋을 활용하여, 베리에이션 및...",채용 절차\n🚢 합류 여정을 안내드려요\n• 서류 전형 > 실무 인터뷰 > 컬쳐핏 ...,복지 및 혜택\n✔️ 몰입을 위해 지원합니다\n서로에 대한 신뢰를 바탕으로 설계된 ...,"회사위치\n서울시 강남구 테헤란로 44길 8, 12층 & 13층 (아이콘역삼빌딩)"
